Running LLaMa 2 on a collab.

1. Read LLaMa 2 licence https://llama.meta.com/llama-downloads/

2. Request access huggingface LLaMa-2

3. Create Huggingface token

4. Install libraries
  *   transformers
  *   bitsandbytes (for 4-bit quantization)
  *   accelerate (for device_map)

5. Download model

6. Run Prompt!




In [1]:
!pip install transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00


Note that to run the following code, you must have got access to Llama 2's weights and have an access token from Hugging Face. You can find instructions on the model cards on the hugging face hub: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)



config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [19]:
prompt = """classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]

Great product, 10/10:
Didn't work very well:
Super helpful, worth it:
I'm on the free tier, It doesnt work!:
I'm a pro user, This product was ok:
This product was bad:
I want a refund:
I want a refund and I'll tell everyone this is bad:
It's exceptionally bad:
Dog:"""

model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=128)

print(tokenizer.decode(output[0], skip_special_tokens=True))

classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]

Great product, 10/10:
Didn't work very well:
Super helpful, worth it:
I'm on the free tier, It doesnt work!:
I'm a pro user, This product was ok:
This product was bad: 
I want a refund:
I want a refund and I'll tell everyone this is bad:
It's exceptionally bad:
Dog:

Reviews:

Great product, 10/10: Promoter
Didn't work very well: Neutral
Super helpful, worth it: Promoter
I'm on the free tier, It doesn't work!: Churn risk
I'm a pro user, This product was ok: Neutral
This product was bad: Detractor
I want a refund: Churn risk
I want a refund and I'll tell everyone this is bad: Detractor
It's exceptionally bad: Detractor
Dog: Random



In [42]:
prompt = """Classify the following reviews.
Great product, 10/10: Promoter ###
Didn't work very well: Detractor ###
Super helpful, worth it: Promoter ###
I'm on the free tier, It doesnt work!: Churn Risk ###
I'm a cat: Random ###
I'm a pro user, This product was ok:
This product was bad:
I want a refund and I'll tell everyone this is bad:
It's exceptionally bad:
Dog:"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt):])

Classify the following reviews.
Great product, 10/10: Promoter ###
Didn't work very well: Detractor ###
Super helpful, worth it: Promoter ###
I'm on the free tier, It doesnt work!: Churn Risk ###
I'm a cat: Random ###
I'm a pro user, This product was ok:
This product was bad:
I want a refund and I'll tell everyone this is bad:
It's exceptionally bad:
Dog: Random ###
I don't know how to use it:
I'm so disappointed:
I'm a beginner, it's easy to use:
I'm a beginner, it's hard to use:
I'm a power user, it's the best:
I'm a power user, it's the worst:
I'm a power user, it's the best for me:
I'm a power user, it's the worst for me:
I'm a power user, it's just ok:
I'm a power user, it's great:
I'm a power user, it's terrible:



No input tokens:

 Random ###
I don't know how to use it:
I'm so disappointed:
I'm a beginner, it's easy to use:
I'm a beginner, it's hard to use:
I'm a power user, it's the best:
I'm a power user, it's the worst:
I'm a power user, it's the best for me:
I'm a power us

In [33]:
prompt = """classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]. Output with the same format in plain text. Do not include any other formatting.[\INST]

review: Super helpful, worth it. classification: Promoter
review: I'm on the free tier, It doesnt work! classification: Churn Risk
review: I'm a cat classification: Random
review: I'm a pro user, This product was ok  classification:
review: This product was bad classification: I want a refund:
review: I want a refund and I'll tell everyone this is bad classification:
review: It's exceptionally bad classification:
review: Dog classification: """
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt):])

classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]. Output with the same format in plain text. Do not include any other formatting.[\INST]

review: Super helpful, worth it. classification: Promoter
review: I'm on the free tier, It doesnt work! classification: Churn Risk
review: I'm a cat classification: Random
review: I'm a pro user, This product was ok  classification:
review: This product was bad classification: I want a refund:
review: I want a refund and I'll tell everyone this is bad classification:
review: It's exceptionally bad classification:
review: Dog classification: Random

Please let me know if you need any additional information.


No input tokens:

Random

Please let me know if you need any additional information.


In [40]:
prompt = """<s>[INST]review: Super helpful, worth it.[/INST]classification: Promoter
[INST]review: I'm on the free tier, It doesnt work! [/INST]classification: Churn Risk
[INST]review: I'm a cat [/INST]classification: Random
[INST]review: I'm a pro user, This product was ok  [/INST]classification: Churn risk
[INST]review: This product was bad [/INST]classification: Detractor
[INST]review: I want a refund and I'll tell everyone this is bad [/INST]classification: """
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt)-3:])

[INST]review: Super helpful, worth it.[/INST]classification: Promoter
[INST]review: I'm on the free tier, It doesnt work! [/INST]classification: Churn Risk
[INST]review: I'm a cat [/INST]classification: Random
[INST]review: I'm a pro user, This product was ok  [/INST]classification:
[INST]review: This product was bad [/INST]classification: Detractor
[INST]review: I want a refund and I'll tell everyone this is bad [/INST]classification: Detractor

I hope this helps! Let me know if you have any other questions.


No input tokens:

Detractor

I hope this helps! Let me know if you have any other questions.


In [41]:
prompt = """<s>[INST]<<SYS>classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]. Output with the same format in plain text. Do not include any other formatting. <</SYS>>
review: Super helpful, worth it.[/INST] classification: Promoter
[INST]review: I'm on the free tier, It doesnt work! [/INST]classification: Churn Risk
[INST]review: I'm a cat [/INST]classification: Random
[INST]review: I'm a pro user, This product was ok  [/INST]classification: Churn risk
[INST]review: This product was bad [/INST]classification: Detractor
[INST]review: I want a refund and I'll tell everyone this is bad [/INST]classification: """
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs,max_new_tokens=150)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print(decoded)
print("\n\nNo input tokens:\n")
print(decoded[len(prompt)-3:])

[INST]<<SYS>classify the following reviews into these categories:[Promoter, Neutral, Churn risk, Churned, Detractor, Random]. Output with the same format in plain text. Do not include any other formatting. <</SYS>>
review: Super helpful, worth it.[/INST] classification: Promoter
[INST]review: I'm on the free tier, It doesnt work! [/INST]classification: Churn Risk
[INST]review: I'm a cat [/INST]classification: Random
[INST]review: I'm a pro user, This product was ok  [/INST]classification:
[INST]review: This product was bad [/INST]classification: Detractor
[INST]review: I want a refund and I'll tell everyone this is bad [/INST]classification: Churned


No input tokens:

Churned
